In [1]:
import os
import shutil
import warnings
from creds import OPENAI_KEY
from application_generator import JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA

# suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

llm = ChatGPT(
    api_key=OPENAI_KEY,
    model="gpt-4o",
    system_prompt=RESUME_WRITER_PERSONA,
    max_output_tokens=None,
    temperature=0.7
)

generator = JobApplicationBuilder(
    llm=llm,
)

job_content_path = r"input_data/Netflix_JR30640.html"
resume_pdf_path = r"input_data/full_resume_contents_20241226.pdf"
#resume_pdf_path = r"input_data/Grad School Resume.pdf"
job_details_dict, job_details_filepath = generator.extract_job_content(job_content_path)
resume_dict = generator.user_data_extraction(user_data_path=resume_pdf_path)
resume_details_dict, resume_details_filepath = generator.generate_resume_json(job_content=job_details_dict,
                                                                              user_data=resume_dict)

resume_latex, resume_tex_path = generator.resume_json_to_resume_tex(resume_details=resume_details_dict)
print("Done generating resume tex file")

cover_letter, cover_letter_txt_path = generator.generate_cover_letter(job_details=job_details_dict,
                                                                      user_data=resume_details_dict,
                                                                      need_pdf=False)

# check that the fonts in the resume tex are available and flag the ones not installed
resume_tex_fonts = LatexToolBox.extract_tex_font_dependencies(resume_tex_path)
font_statuses = LatexToolBox.check_fonts_installed(resume_tex_fonts)
if not all([v for k, v in font_statuses.items()]):
    [print(f"{k} not installed") for k, v in font_statuses.items() if not v]

/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/aaronrdankert/projects/job_hunter_toolbox/utils.py:195: DeprecationWarning: invalid escape sequence '\d'
  if '\documentclass{resume}' in open(tex_file_path).read():
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydev_ipython/matplotlibtools.py:59: MatplotlibDeprecationWarning: The interactive_bk attribute was deprecated in Matplotlib 3.9 and will be removed in 3.11. Use ``matplotlib.backends.backend_registry.list_builtin(matplotlib.backends.BackendFilter.INTERACTIVE)`` instead.
  from matplotlib.rcsetup import interactive_bk, non_interactive_bk  # @UnresolvedImport
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.1

Job Details JSON generated at: output_files/Adobe/2025InternDataS_20241224161043/JD.json

Fetching user data...


/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The name or title of the project.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The type or category of the project, such as hackathon, publication, professional, and academic.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default va


Generating Resume Details...
Processing Resume's WORK_EXPERIENCE Section...
Processing Resume's PROJECTS Section...


/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The name or title of the project.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The type or category of the project, such as hackathon, publication, professional, and academic.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default va

Processing Resume's SKILL_SECTION Section...
Processing Resume's EDUCATION Section...
Processing Resume's CERTIFICATIONS Section...
Processing Resume's ACHIEVEMENTS Section...
Done generating resume tex file
Cover Letter generated at:  output_files/Adobe/2025InternDataS_20241224161043/cover_letter.txt


Matplotlib is building the font cache; this may take a moment.


output_files/Adobe/2025InternDataS_20241224161043/resume.tex not installed


Edit the resume latex before running the next cell.

In [2]:
LatexToolBox.compile_latex_to_pdf(tex_file_path=resume_tex_path)
print(f"Resume PDF is saved at {resume_tex_path.replace('.tex', '.pdf')}")

cover_letter_pdf_path = cover_letter_txt_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(cover_letter, cover_letter_pdf_path)
print(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

/Users/aaronrdankert/projects/job_hunter_toolbox/utils.py:195: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/aaronrdankert/projects/job_hunter_toolbox/output_files/Adobe/2025InternDataS_20241224161043/resume.tex' mode='r' encoding='UTF-8'>
  if '\documentclass{resume}' in open(tex_file_path).read():


PDF successfully saved to /Users/aaronrdankert/projects/job_hunter_toolbox/output_files/Adobe/2025InternDataS_20241224161043
Resume PDF is saved at output_files/Adobe/2025InternDataS_20241224161043/resume.pdf
Cover Letter PDF is saved at output_files/Adobe/2025InternDataS_20241224161043/cover_letter.pdf


In [3]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

'output_files/Adobe/DataScienceEngi_20241220172036/_1'

In [ ]:
from utils import LatexToolBox
fonts_required, _ = LatexToolBox.extract_tex_font_dependencies('output_files/Adobe/InternMachineLe_20241209195126/resume.tex')
font_dict = LatexToolBox.check_fonts_installed(fonts_required)
print(font_dict)